<a href="https://colab.research.google.com/github/adolfoguimaraes/datascience/blob/main/code/08_laboratorio_avaliacao_aprendizado_supervisionado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizado Supervisionado

Vamos trabalhar com mais uma base e testar diferentes algortimos de Machine Learning. No entanto, nosso foco vai ser olhar as métricas de avaliação estudadas em sala de aula.

In [ ]:
import pandas as pd
from tabulate import tabulate
from sklearn.model_selection import cross_validate

import warnings
warnings.filterwarnings('ignore')


## Base dos Dados 

Vamos utilizar a base de dados disponível nesse link: https://archive.ics.uci.edu/ml/datasets/spambase. A base está disponível na pasta `datasets` com o nome `SpamDataset.csv`. A versão deste repositório já está com os nomes das colunas. No repositório original, essas informações estão em arquivos separados. Não foi feita nenhuma modificação nos dados, apenas a inclusão no nome das colunas. 

In [ ]:
data_ = pd.read_csv("../datasets/SpamDataset.csv")
data_

Antes de começarmos a definir nossa estratégia de treinamento, vamos olhar como estão distribuídos esses dados. 

In [ ]:
pd.DataFrame(data_['classe'].value_counts(normalize=True))

Isso indica que 60% das instâncias estão classificadas como não-spam e 40% como Spam. Existe um desbalanceamento dos dados e precisamos pensar em qual classe vamos levar em consideração na hora de treinar nossos modelos.

## Definindo o Treinamento

### Separação Dados e Classe

In [ ]:
X = data_[data_.columns[:-1]]
y = data_[data_.columns[-1]]

### Métodos utilizados

Vamos criar uma estratégia de treinamento de diferentes modelos de Machine Learning a tarefa de classificação de SPAM. Vamos testar os seguintes modelos e parâmetros:

* **KNN:** Variar o valor de `n_neighbors` de 1 a 15. 
* **SVM:** Usar o kernel rbf e variar os parâmetros `C` e `gamma` com os valores [0.01, 0.1, 1, 10, 100] (para o `C`) e [0.01, 0.1, 1] para o `gamma`.
* **Árvore de Decisão:** Será variado o `max_depth` com os valores [None, 1, 5, 10, 20, 30]. 
* **Regressão Logística:** Vamos fixar o solver em `liblinear` e variar `C` com os valores: [0.01, 0.1, 1, 10, 100].
* **Radom Forest:** Vamos varia o `n_estimators` e o `max_depth` com so valores [10, 50, 100, 200] (para o `n_estimators`) e [None, 1, 5, 10, 20, 30] (para o `max_depth`),

In [ ]:
# Instanciando os métodos utilizados e o método para aplicar GridSearch

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

### Parâmetros

Foram escolhidas apenas algumas variações de parâmetro com base no que foi discutido em sala.

In [ ]:
gridsearch_param_knn = {'n_neighbors': list(range(1,16))}
gridsearch_param_svm = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}
gridsearch_param_dt = {'max_depth': [None, 1, 5, 10, 20, 30]}
gridsearch_param_lr = {'C': [0.01, 0.1, 1, 10, 100]}
gridsearch_param_rf =  {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 1, 5, 10, 20, 30]}


### Execução do Treinamento

In [ ]:

# Método que aplica o GridSearch com os parâmetros definido e retorna o melhor resultado de acordo com a métrica de interesse (score)

def fit_grid_search(model, params, score, X, y):

    gridsearch_model = GridSearchCV(model,param_grid=params, cv=5, scoring=['accuracy','precision','recall','f1'], refit=score)

    gridsearch_model.fit(X, y)


    print("Melhores parâmetros:", gridsearch_model.best_params_)
    print("Melhor score (%s): %f" % (score,gridsearch_model.best_score_ ))
    
    
    dt_frame = pd.DataFrame(gridsearch_model.cv_results_)

    dt_frame = dt_frame.query("rank_test_" + score + " == 1")
    dt_frame = dt_frame[['mean_fit_time','std_fit_time','mean_score_time','std_score_time','mean_test_accuracy','mean_test_precision','mean_test_recall','mean_test_f1']]

    return dt_frame


In [ ]:
# Instanciando os métodos com os parâmetros padrões ou fixos.
knn_ = KNeighborsClassifier()
svm_ = SVC(kernel='rbf')
dt_ = DecisionTreeClassifier()
lr_ = LogisticRegression(solver='liblinear')
rf_ = RandomForestClassifier()

#### KNN

In [ ]:
fit_grid_search(knn_, gridsearch_param_knn, 'accuracy', X, y)

#### Árvore de Decisão

In [ ]:
fit_grid_search(dt_, gridsearch_param_dt, 'accuracy', X, y)

#### Regressão Logística

In [ ]:
fit_grid_search(lr_, gridsearch_param_lr, 'accuracy', X, y)

#### Random Forest

In [ ]:
fit_grid_search(rf_, gridsearch_param_rf, 'accuracy', X, y)

#### SVM

In [ ]:
fit_grid_search(svm_, gridsearch_param_svm, 'accuracy', X, y)